In [ ]:
%%capture

!python -m nbconvert --to html 'Intro.ipynb' --no-input
!open 'Intro.html'

In [ ]:
%%capture

!python -m nbconvert --to slides 'Intro.ipynb' --reveal-prefix reveal.js --no-input
!open 'Intro.slides.html'

Lewisham's 2022 Ward Boundaries
==
Lewisham's ward boundary [changes](https://consultation.lgbce.org.uk/node/17020) for the 2022 elections. 

_This document was produced by the Lewisham Insight & Delivery team._

The new ward boundaries
--
* The boundaries of 2 wards will stay the same (Telegraph Hill and Forest Hill) – all others will change
* 2 wards will no longer exist – New Cross and Whitefoot
* We will have 3 brand new wards – Deptford, Hither Green and New Cross Gate
* 16 wards will have 3 councillors
* 3 wards will have only 2 councillors – Bellingham, Lewisham Central and New Cross Gate
* The total number of councillors will remain the same as previously

In [ ]:
%%capture
try:
    import liad
except ImportError as e:
    %pip install git+https://github.com/lb-lewisham/liad.git

In [ ]:
%%capture
try:
    import geopandas
except ImportError as e:
    %pip install geopandas


In [ ]:
%%capture
try:
    import natsort
except ImportError as e:
    %pip install natsort


In [ ]:
%%capture
from liad import colab
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import IPython
import altair as alt
import numpy

In [ ]:
if colab.in_colab():
    from google.colab import drive
    drive.mount("/content/gdrive")
    project_dir = "/content/gdrive/MyDrive/WardProfiles"
else:
    project_dir = "/Volumes/GoogleDrive/My Drive/WardProfiles"

In [ ]:
%%capture
import sys
sys.path.insert(1, project_dir + '/utils')

import inputs

In [ ]:
%%capture
import importlib
importlib.reload(inputs)

In [ ]:
lewisham = gpd.read_file('https://gist.githubusercontent.com/joe-liad/e66e2ec493ce3de692595b64eeb27b99/raw/bfaafeec350dfe4e5a20866a4660a188f8e6df7d/lewisham-wards.geojson')
# apply correct winding order to the polygons
from shapely.ops import orient # version >=1.7a2
lewisham.geometry = lewisham.geometry.apply(orient, args=(-1,))
lewisham.drop(columns=['OBJECTID','No_of_coun','Current_el','Forecast_e','SHAPE_Leng'], inplace=True)

In [ ]:
lewisham.explore(
  style_kwds={'stroke':True, 'color':'#fee05c', 'fillColor':'#00b7eb', 'weight':1}, 
  attr='Lewisham Insight & Delivery | Map tiles by <a href="http://stamen.com">Stamen</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> | Map data © <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', 
  highlight=True, tooltip=['Name'], 
  tiles='Stamen Toner'    
)

In [ ]:
def melted(df, id_vars, var_name, value_name):
  return (
      df
      .reset_index()
      .melt(id_vars=id_vars, var_name=var_name, value_name=value_name
    )
  )

In [ ]:
def joined (gdf, df, join_on_gdf, join_on_df):
  return inputs.join_gdf_df(gdf, df,join_on_gdf, join_on_df)

In [ ]:
def facet_map(df, facet, title, color_by, tooltip_title, orient="left", fmt=".2f"):
  facets = df[facet].unique()
  facets = numpy.delete(facets, np.where(facets == 'index'))
  return alt.concat(*(
      alt.Chart(df[df[facet] == facet_level], title=facet_level).mark_geoshape().encode(
        color=alt.Color(color_by,legend=alt.Legend(title=title, orient=orient), sort="descending"),
        tooltip=["Name:N", alt.Tooltip(facet, title=facet), alt.Tooltip(color_by, title=tooltip_title, format=fmt)]
      ).properties(
        width=200, height=200
      )
      for facet_level in facets
    ), columns=len(facets)
  )  

# Population in 2019

In [ ]:
alt.Chart(inputs.join_gdf_df(lewisham, inputs.pop_totals, 'Ward_name', 'Name')).mark_geoshape().encode(
        color=alt.Color('Total', sort="descending", legend=alt.Legend(title='Population', orient='right')),
        tooltip=["Name:N", "Total:Q", "SHAPE_Area:Q"]
      ).properties(
        width=400, height=200
      )

## Population by age group

In [ ]:
df = melted(df=inputs.pop_age_groups, id_vars=['Name'], var_name='age_group', value_name='age')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='age_group', title="Percent of ward population", color_by='age:Q', tooltip_title='Percent')

## Population by age group (female)

In [ ]:
df = melted(df=inputs.pop_female, id_vars=['Name'], var_name='age_group', value_name='age')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='age_group', title="Percent of ward population", color_by='age:Q', tooltip_title='Percent')

## Population by age group (male)

In [ ]:
df = melted(df=inputs.pop_male, id_vars=['Name'], var_name='age_group', value_name='age')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='age_group', title="Percent of ward population", color_by='age:Q', tooltip_title='Percent')

In [ ]:
%%capture
import importlib
importlib.reload(inputs)

## Languages spoken

### English as main language

In [ ]:
df = melted(df=inputs.english_proficiency[['English is main language', 'Name']], id_vars=['Name'], var_name='proficiency', value_name='language')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='proficiency', title="Percent of ward population", color_by='language:Q', tooltip_title='Percent')

In [ ]:
df = melted(df=inputs.english_proficiency[['Can speak English well or very well', 'Name']], id_vars=['Name'], var_name='proficiency', value_name='language')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='proficiency', title="Percent of ward population", color_by='language:Q', tooltip_title='Percent')

In [ ]:
df = melted(df=inputs.english_proficiency[['Cannot speak English well or at all',	'Name']], id_vars=['Name'], var_name='proficiency', value_name='language')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='proficiency', title="Percent of ward population", color_by='language:Q', tooltip_title='Percent')

### Other languages
The following maps show wards where more than 1% of residents speak a non-english 
language as their **main language**

In [ ]:
%%capture --no-display
df = melted(df=inputs.main_language, id_vars=['Name'], var_name='main_lang', value_name='language')
df = df[df.main_lang != 'index'][(df.language>=1) & (df.language<50)]
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
df.drop(columns=['index','SHAPE_Area'], inplace=True)
facet_map(df, facet='main_lang', title="Percent of ward population", color_by='language:Q', tooltip_title='Percent')

## Multiple deprivation

_Please note: the mapping of statistical LSOA to administrative Ward is not perfectly aligned, so this data should not be considered as more than **indicative**_

In [ ]:
# NOTE THAT THIS IS NOT WELL SPATIALLY FITTED DATA
df = inputs.imd.groupby('Name').mean()
df = df[['IMD_decile']]
df = melted(df, id_vars=['Name'], var_name='key', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='key', title="IMD", color_by='val:Q', tooltip_title='Average decile')


## Benefits

### Female and male

In [ ]:
df = melted(df=inputs.benefits, id_vars=['Name'], var_name='Month', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Month', title="Recipients in ward", color_by='val:Q', tooltip_title='Count')

### Female

In [ ]:
df = melted(df=inputs.benefits_female, id_vars=['Name'], var_name='Month', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Month', title="Recipients in ward", color_by='val:Q', tooltip_title='Count')

### Male

In [ ]:
df = melted(df=inputs.benefits_male, id_vars=['Name'], var_name='Month', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Month', title="Recipients in ward", color_by='val:Q', tooltip_title='Count')

# Country of birth
## England

In [ ]:
df = melted(df=inputs.country_of_birth.loc[:, inputs.country_of_birth.columns.str.contains('Name|England|Total', regex=True)], id_vars=['Name'], var_name='country', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='country', title="Recipients in ward", color_by='val:Q', tooltip_title='Count', fmt='0.0f')

## Not England

In [ ]:
df = melted(df=inputs.country_of_birth.loc[:, ~inputs.country_of_birth.columns.str.contains('Total|England|otherwise', regex=True)], id_vars=['Name'], var_name='country', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='country', title="Residents in ward", color_by='val:Q', tooltip_title='Count', fmt='0.0f')

In [ ]:
df = inputs.educational_attainment
df = melted(df=df.loc[:, ~df.columns.str.contains('Total', regex=True)], id_vars=['Name'], var_name='qualification', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='qualification', title="Recipients in ward", color_by='val:Q', tooltip_title='Count', fmt='0.0f')


# Employment

## Occupation status

In [ ]:
df = inputs.economic_activity
df = melted(df=df.loc[:, ~df.columns.str.contains('Total', regex=True)], id_vars=['Name'], var_name='Status', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Status', title="Percent", color_by='val:Q', tooltip_title='Percent', fmt='0.2f')


## Hours worked

In [ ]:
df = inputs.hours_worked

cols = df.columns[~inputs.hours_worked.columns.str.contains('Name', regex=True)]
df[cols] = df[cols].div(df['Total'], axis=0)*100

df = melted(df=df.loc[:, ~df.columns.str.contains('Total', regex=True)], id_vars=['Name'], var_name='Occupation', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Occupation', title="Count", color_by='val:Q', tooltip_title='Percent', fmt='0.0f')


## Occupation groups

In [ ]:
df = inputs.occupation_share
df = melted(df=df.loc[:, ~df.columns.str.contains('Total', regex=True)], id_vars=['Name'], var_name='Hours worked', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Hours worked', title="Percent", color_by='val:Q', tooltip_title='Percent', fmt='0.0f')


In [ ]:
def natsort_columns(columns):
  from natsort import index_natsorted
  df = pd.DataFrame(columns)
  return df.sort_values(
    by=0,
    key=lambda x: np.argsort(index_natsorted(df[0]))
  )


In [ ]:
sorted_columns = natsort_columns(inputs.occupation_minor_groups.columns)
df = inputs.occupation_minor_groups.iloc[:,sorted_columns[0].index]

cols = df.columns[~df.columns.str.contains('Name', regex=True)]
df[cols] = df[cols].div(df['Total'], axis=0)*100

df.set_index('Name', inplace=True)


In [ ]:
def professions(df,start,end):
  df = melted(df=df.iloc[:, start:end], id_vars=['Name'], var_name='Profession', value_name='val')
  df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
  return facet_map(df, facet='Profession', title="Percent", color_by='val:Q', tooltip_title='Percent', fmt='0.0f')

In [ ]:
professions(df, 0, 9)

In [ ]:
professions(df, 10, 19)

In [ ]:
professions(df, 20, 29)

In [ ]:
professions(df, 30, 39)

In [ ]:
professions(df, 40, 49)

In [ ]:
professions(df, 50, 59)

In [ ]:
professions(df, 60, 69)

In [ ]:
professions(df, 70, 79)

In [ ]:
professions(df, 80, 89)

In [ ]:
professions(df, 90, 99)

In [ ]:
professions(df, 100, 109)

In [ ]:
professions(df, 110, 119)

In [ ]:
professions(df, 120, 124)

## Ethnicity

In [ ]:
df = inputs.ethnicity

df = melted(df=df.loc[:, ~df.columns.str.contains('Total', regex=True)], id_vars=['Name'], var_name='Ethnicity', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Ethnicity', title="Count", color_by='val:Q', tooltip_title='Percent', fmt='0.0f')


## Fuel poverty

In [ ]:
grouped = inputs.fuel_poverty.groupby(by='Name', axis=0)
df = grouped.mean()
df = melted(df=df.loc[:, ~df.columns.str.contains('Total|Number', regex=True)], id_vars=['Name'], var_name='Poverty', value_name='val')
df = joined(lewisham, df, join_on_gdf='Ward_name', join_on_df='Name')
facet_map(df, facet='Poverty', title="Count", color_by='val:Q', tooltip_title='Percent', fmt='0.0f')
